# <p style="text-align: center;"> <span style="color:yellowgreen"> Running the MM simulator </span></p>

In this notebook, we will use what we have learned about ROS in the exercises to start to control the duckiebot in simulation. 

## See what the Duckiebot Sees

1. As in the exercies, we will launch the vnc container in the browser by running following command in the terminal **of your laptop** (not the terminal in VSCode!) except this time with the `--sim` argument. 

**Note:** Make sure you are in your `lab1` directory, using the `cd` command if needed:





In [ ]:
dts code workbench --recipe ../lab1-recipe --sim

**Note 1:** You will need to stop any previous vnc container with CTRL-C and then execute this with the `--sim` option.

**Note 2:** This may take some time to run the first time because it needs to download the docker image of the Duckietown simulation environment. 

2. Open the vnc container with the link generated after running the `dts code workbench`command

3. Once you open the vnc container, as before, you should see the same icons on the left hand side.  
Double-click on the one that says `RQT Image ...`, clicking on this opens the ROS utility.  
In the top left scroll down bar of the `rqt_image_view` window you should see only one option: `/agent/camera_node/image/compressed` if you select it you will see the output from the camera on the virtual Duckiebot.


<br><img src="../assets/lab1-images/vnc-image-sim.png" style="width: 30%; height: auto;"></br>




## Manually driving the MM in the simulator

1. In the vnc container that you have opened above:  
    a. Double click on the `Joystick` icon.  
    b. You can try clicking on the directions or using the arrow keys on your keyboard.  
    c. You should see the corresponding direction on the joystick turn green, but you **won't** see the robot moving in the simulation (assuming you still have the`rqt_image_view` window open from the previous step).

2. Open the `LXTerminal` in the vnc container. 
Go the the catkin_ws to build and activate it



In [ ]:
cd /code/catkin_ws
catkin build
source devel/setup.bash

3. Run the dt-joystick-demo package in the `LXterminal` you have opened

In [ ]:
rosrun dt-joystick-demo dt-joystick-demo-node.py

Now when you use your arrow keys on the joystick controller, you should see the duckiebot move in the simulation. If it still is not moving, make sure you are in the joystick window and the arrows light up when you press your arrow keys.

**NOTE:** You can examine the topic used to make the MM move in the simulation by checking the ROS node list using the debugging tools in a new terminal and look for the joystick demo node. To make a new terminal tab, select **File** in the top left of the LXTerminal window and select **New Tab**.

**NOTE:** While you are still running the simulator. 
Check the other topics available and their information (which message do they take) using the rostopic commands.
You should see the following topics.

```
/agent/camera_node/camera_info
/agent/camera_node/image/compressed
/agent/coordinator_node/intersection_go
/agent/episode_start
/agent/joy
/agent/led_emitter_node/led_pattern
/agent/left_wheel_encoder_node/tick
/agent/right_wheel_encoder_node/tick
/agent/wheels_driver_node/emergency_stop
/agent/wheels_driver_node/wheels_cmd
/rosout
/rosout_agg
```

### The Task Ahead

Our objective for the rest of this exercise will be to create a ROS node that publish messages to the `/agent/wheels_driver_node/wheels_cmd` topic so that the MM moves around the block. 

You can close the vnc container for now by pressing CTRL+C in you hostOS terminal where the vnc is running.

## Create a New Package

Go to the vscode container now and create a new package for the task of moving the MM around the block.  
Run the dts code editor command in a terminal in the hostOS in the lab1 folder. 

In [ ]:
dts code editor --recipe ../lab1-recipe

### Package Instructions:

- Create a package as we have done it in Exercise 2 as well as a node.  
- Name the package as `lab1` and the node `mmdriver.py`  
- The package will have the following dependencies: `std_msgs sensor_msgs duckietown_msgs rospy`  

Run the following command in the folder you want the package to be stored in: 

In [ ]:
catkin_create_pkg lab1 std_msgs sensor_msgs duckietown_msgs rospy

- Remember to make `mmdriver.py` an executable file by using the chmod +x command

## Writing the ROS Node

The code below is a template for the ROS Node with detailed explainations.

In [ ]:
#!/usr/bin/env python3

import rospy
import os
from duckietown_msgs.msg import WheelsCmdStamped # Import the message for the wheel command

class CLASSNAME():#CHANGE CLASSNAME to the name of your class
    def __init__(self):
        
        veh_name = os.environ['VEHICLE_NAME']
		# USING PARAMETER TO GET THE NAME OF THE VEHICLE
        # THIS WILL BE USEFUL TO SPECIFY THE NAME OF THE TOPIC


		# INITIALIZE YOUR VARIABLES HERE (SUBSCRIBERS OR PUBLISHERS)

    def YOUR_FUNCTION(self): # CHANGE TO THE NAME OF YOUR FUNCTION
        #WRITE THE CODE TO MAKE THE MM GO AROUND THE BLOCK

if __name__ == "__main__": ## The main function which will be called when your python script is executed.
    # Initialize the node
    try:
		rospy.init_node('driving')
		drive = CLASSNAME() # Initialize the CLASSNAME (CHANGE with the name of your class), i.e., it will create an object and it will run the __init__ function of your CLASSNAME
		rospy.sleep(30) # Delay to wait enough time for the code to run after the simulator starts 
		# Keep the line above - you might be able to reduce the delay a bit, e.g., 20secs or maybe 10sec, but check if the your node starts after the simulator
		while not rospy.is_shutdown(): # Run ros forever - you can change this as well instead of running forever
			drive.YOUR_FUNCTION() 
			# calling your node function
			rospy.spin() 
	except rospy.ROSInterruptException:
		pass

## General Testing instructions

1. Develop your code in the vscode container (Faster testing and debugging)
	- Test your code in the vscode container first.  
	- Make sure you can run your node in the vscode container before moving to the test it in the simulator.  
	- Add roslogs (prints) in the vscode container to make sure you are publishing the messages correctly. 
	- Leave the vscode container open even when you move for testing in the simulator 


2. Once you are confident with your vscode container testing and debugging, then move to test in the simulator.  
There are instructions below on how to test it in the vnc container with simulator

3. Error discovered while testing in the simulator, e.g., MM does not move due to wrong topic name.
	- Close the vnc container with the simulator (CTRL + C in the hostOS terminal where you opened the vnc container)
	- Go back to the vscode container to fix the error.
	- Once the error is fixed, test in the simulator by reopening the vnc container.
	- The vscode container does not automatically update the code in the vnc container.  
    	- For example, if you fix the error in the vscode container while your vnc container is still open, the fix will not automatically go to the vnc container.
	You will have to close the vnc container and reopen it. 
	For this reason, it is easier to debug as much as you can in the vscode container.



## Testing your code in the simulator

### Option 1 - Using the launcher option when initializing vnc

1. Close any vnc container that might be running.

2. If you made the name of your package and node different from the instructions, you will need to change the **line 11** inside the `lab1-recipe/launchers/lab1.sh`.  
After the rosrun command in line 11, change the package name and node name with the name of your package and node.

3. In the HostOS terminal, start the vnc container with the --sim and the --launcher lab1 options:

In [ ]:
dts code workbench --recipe ../lab1-recipe --sim --launcher lab1


4. Once the vnc container starts, you can open the `rqt_image_view`.  
Although it is a little unclear for now, this `--launcher lab1` will run your node (as long as you called the package and the node exactly what was specified above). 

If all went well, you should see that your node is making the virtual MM move.
If not, go back to the HostOS terminal where you ran `dts code workbench` to see if there are any errors reported by your code.
If there are any errors, fix them in the vscode container.

**Note:** The simulation takes some time to start. 
In order for your node to start after the simulation starts, add at least 20 seconds of delay to the start before running the main parts of your nodes.

5. You can also observe the simulation by visiting the following link (It might take some time to see something appear-keep refreshing):



In [ ]:
http://localhost:8090/


### Option 2 - Using the LXTerminal in vnc

1. Close any vnc container that might be running.

2. In the HostOS terminal, start the vnc container with the --sim option

In [ ]:
dts code workbench --recipe ../lab1-recipe --sim

4. Open the vnc container and open the `LXTerminal` and the `rqt_image_view`

5. In the LXTerminal, go to the `code/catkin_ws` directory by using the cd command

6. In the catkin_ws folder, build the catkin workspace and activate it (catkin build and source commands)

7. Run your launch file with: roslaunch filename.launch

If all went all, you should see that your node is making the virtual MM move.
If not, go back to the LXTerminal to see any errors reported.
If there are any errors, close the vnc and fix them in the vscode container.

**Note:** The simulation takes some time to start. 
Using the LXTerminal does not need too much delay time before starting your nodes. Having at least 5 seconds of delay should be enough.

8. You can also observe the simulation visiting the following link below.


In [ ]:
http://localhost:8090/

## Getting a video from your simulation

If you let your simulation run for a couple of minutes, all simulation episodes will terminate.  
One way to get the video faster is by driving the MM out of bounds. 
You run open the joystick and run the LXTerminal `rosrun dt-joystick-demo dt-joystick-demo-node.py`

The following message in the hostOS terminal should appear.

<br><img src="../assets/lab1-images/MM-sim-end-of-episodes.png" style="width: 40%; height: auto;"></br>

After all episodes are done, you can close the vnc container in the HostOS terminal.  
In the bottom, a link for a tmp folder will appear as in the image below.   
Open the link, it should open a new VSCODE in the hostOS.  
Or you can use the folder directory and go to that path.  

<br><img src="../assets/lab1-images/MM-sim-video-link.png" style="width: 40%; height: auto;"></br>

You will be able to access the video in this folder.  

<br><img src="../assets/lab1-images/MM-sim-video-folder.png" style="width: 40%; height: auto;"></br>
